## Unit 11. 非線性模型回歸 課堂作業
- 課程編號: CHE-AI-114
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: 莊曜禎 助理教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 環境設定

In [ ]:
from pathlib import Path
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P3_Unit11_HW'
SOURCE_DATA_DIR = 'HW'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_3' / 'Unit11'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")

---
### **I. 練習題: 建立並比較多種非線性回歸模型**
 - 使用下面觸媒反應製程模擬數據集
 - 建立 Unit 11 學習的所有非線性模型
 - 進行模型訓練與綜合評估比較

### **0. 數據生成**
生成觸媒反應系統的模擬數據，包含複雜的非線性關係

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats.qmc import LatinHypercube

# 設定隨機種子
np.random.seed(42)

# 使用拉丁超立方採樣生成特徵數據
n_samples = 500
n_features = 3
sampler = LatinHypercube(d=n_features, seed=42)
sample = sampler.random(n=n_samples)

# 定義特徵範圍
bounds = np.array([
    [100, 300],     # 反應溫度 (°C)
    [1, 50],        # 反應壓力 (bar)
    [0.5, 5.0]      # 觸媒用量 (wt%)
])

# 縮放到實際範圍
X = bounds[:, 0] + sample * (bounds[:, 1] - bounds[:, 0])

df = pd.DataFrame(X, columns=['Temperature', 'Pressure', 'Catalyst_Loading'])

# 生成目標變數(轉化率) - 複雜非線性關係
# 包含 Arrhenius 溫度效應、壓力飽和效應、觸媒用量飽和、交互作用
def catalyst_reaction_model(T, P, Cat, noise_level=3.5):
    """
    模擬觸媒反應轉化率
    
    參數:
        T: 溫度 (°C)
        P: 壓力 (bar)
        Cat: 觸媒用量 (wt%)
        noise_level: 噪聲標準差
    """
    # 標準化溫度 (用於 Arrhenius)
    T_norm = (T - 200) / 100
    
    # Arrhenius 溫度效應 (溫度越高反應越快)
    temp_effect = 36 * (1 - np.exp(-0.8 * T_norm))
    
    # 壓力飽和效應 (壓力增加但有飽和)
    pressure_effect = 23 * (1 - np.exp(-0.1 * P))
    
    # 觸媒用量飽和效應
    catalyst_effect = 26 * (1 - np.exp(-0.6 * Cat))
    
    # 交互作用: 溫度與壓力
    interaction_TP = 0.015 * T_norm * P
    
    # 交互作用: 觸媒與溫度
    interaction_CT = 0.5 * Cat * T_norm
    
    # 高溫失活懲罰 (溫度過高觸媒失活)
    deactivation = -0.008 * np.maximum(0, T - 250)**2
    
    # 基礎轉化率
    base_conversion = 15
    
    # 總轉化率
    conversion = (base_conversion + 
                 temp_effect + 
                 pressure_effect + 
                 catalyst_effect + 
                 interaction_TP + 
                 interaction_CT + 
                 deactivation)
    
    # 添加高斯噪聲
    noise = np.random.normal(0, noise_level, len(T))
    conversion = conversion + noise
    
    # 限制在合理範圍
    conversion = np.clip(conversion, 0, 100)
    
    return conversion

# 生成轉化率
conversion = catalyst_reaction_model(
    df['Temperature'].values,
    df['Pressure'].values,
    df['Catalyst_Loading'].values,
    noise_level=3.5
)

df['Conversion'] = conversion

# 保存數據
df.to_csv(DATA_DIR/'catalyst_reaction_data.csv', index=False, encoding='utf-8-sig')
print("=" * 60)
print("觸媒反應轉化率數據生成完成")
print("=" * 60)
print(f"\n數據集已保存: {DATA_DIR}/catalyst_reaction_data.csv")
print(f"數據形狀: {df.shape}")
print(f"\n轉化率統計:")
print(df['Conversion'].describe())
print(f"\n完整數據集前10筆:")
print(df.head(10))

---
### **1. 數據準備與探索性分析**

**要求**:
- 載入數據並檢查基本統計資訊
- 檢查是否有缺失值
- 繪製目標變數 (Conversion) 的分佈直方圖
- 繪製所有特徵與目標變數的散點圖矩陣
- 計算並視覺化相關係數矩陣
- 數據分割: 訓練集 75%、測試集 25%
- **對 X 和 y 都進行 StandardScaler 標準化**

---
## **2. 建立所有非線性回歸模型**

本單元學習的非線性模型包括:
1. **Polynomial Regression** (多項式回歸)
2. **Decision Tree Regressor** (決策樹)
3. **Gradient Boosting Regressor** (梯度提升樹)
4. **Support Vector Regressor** (支持向量機)
5. **Gaussian Process Regressor** (高斯過程回歸)

請依序完成以下各模型的建立與訓練。

### **2.1 Polynomial Regression (多項式回歸)**

**要求**:
- 使用 `PolynomialFeatures` 將特徵擴展到 2 階或 3 階
- 結合 `Ridge` 回歸進行訓練 (alpha=1.0)
- 記錄訓練時間
- 在測試集上進行預測並計算指標: MAE, RMSE, R²
- **記得對預測結果進行反標準化**

### **2.2 Decision Tree Regressor (決策樹)**

**要求**:
- 設定參數: `max_depth=10`, `min_samples_split=10`, `min_samples_leaf=5`, `random_state=42`
- 訓練模型
- 記錄訓練時間
- 計算測試集指標: MAE, RMSE, R²
- 繪製特徵重要性條形圖
- **記得對預測結果進行反標準化**

### **2.3 Gradient Boosting Regressor (梯度提升樹)**

**要求**:
- 設定參數: `n_estimators=100`, `learning_rate=0.1`, `max_depth=5`, `min_samples_split=5`, `min_samples_leaf=2`, `random_state=42`
- 訓練模型
- 記錄訓練時間
- 計算測試集指標: MAE, RMSE, R²
- 繪製特徵重要性條形圖
- **記得對預測結果進行反標準化**

### **2.4 Support Vector Regressor (支持向量機)**

**要求**:
- 使用 RBF 核函數
- 設定參數: `kernel='rbf'`, `C=100`, `epsilon=0.1`, `gamma='scale'`
- 訓練模型
- 記錄訓練時間
- 計算測試集指標: MAE, RMSE, R²
- **記得對預測結果進行反標準化**

### **2.5 Gaussian Process Regressor (高斯過程回歸)**

**要求**:
- 使用 Matérn 5/2 核函數
- 設定參數: `kernel=C(1.0) * Matern(1.0, nu=2.5)`, `alpha=0.3`, `n_restarts_optimizer=10`, `random_state=42`
- 訓練模型
- 記錄訓練時間
- 計算測試集指標: MAE, RMSE, R²
- **額外**: 計算並繪製預測的不確定性 (標準差)
- **記得對預測結果進行反標準化**

---
## **3. 模型綜合比較與分析**

**要求**:
- 建立一個比較表格，包含所有模型的:
  - 訓練時間
  - 測試集 MAE
  - 測試集 RMSE
  - 測試集 R²
- 繪製所有模型的性能比較圖 (橫向條形圖)
- 針對每個模型繪製「實際值 vs 預測值」散點圖 (subplot)
- 分析並總結:
  - 哪個模型表現最好？為什麼？
  - 哪個模型訓練最快？
  - 哪些模型適合小數據集？哪些適合大數據集？
  - 在化工領域應用中，應該如何選擇模型？

---
## **4. 保存最佳模型**

**要求**:
- 選擇性能最好的模型
- 使用 `joblib` 保存模型到 `OUTPUT_DIR`
- 同時保存 scaler (用於未來預測)

---
## **II. 額外加分作業 (自由繳交)**

- 學生可自由選擇是否繳交加分作業 (下週上課前完成 email 電子檔)
- 分數會加在最後總成績上，每份作業加 0.1 ~ 1.0 分
- 分數加的不多，真的不一定要交加分作業，正常出席上課做好期末報告即可
- 加分作業不一定要全部都做完，想繳交至少完成其中一項實驗即可
- 請務必自行完成！你可以問 AI，問同學，問學長姊，甚至參考以前別人的作業，但一定要自行 [吸收][執行][整理] 結果！
- 不要貼 AI 的答案寄給老師看，你自己看就好
- 如果系統自動比對發現作業內容 (與他人雷同，原創性低，抄襲比例過高)，作業的分數會 0 分
- 如果你直接 100% 複製別人的作業繳交，你會直接被當掉 (提供作業給別人複製的也一樣)
- 老師看你作業要花時間，真的有心想寫加分作業，請好好自己做

### **實驗 1: 超參數調優實驗**

選擇 **Gradient Boosting** 模型，進行系統性的超參數調優。

**Gradient Boosting 超參數組合**:
- `n_estimators`: [50, 100, 200]
- `learning_rate`: [0.01, 0.1, 0.2]
- `max_depth`: [3, 5, 7]

**要求**:
- 使用 `GridSearchCV` 進行網格搜索 (使用 5-fold 交叉驗證)
- 記錄最佳參數組合
- 比較調優前後的性能
- 分析各超參數對模型性能的影響
- 繪製超參數影響圖

### **實驗 2: 不同多項式階數的影響**

比較 Polynomial Regression 使用不同階數的效果。

**要比較的階數**:
- 1 階 (線性)
- 2 階
- 3 階
- 4 階

**要求**:
- 對每個階數:
  - 訓練模型 (使用 Ridge 正則化)
  - 計算測試集 R²、RMSE、MAE
  - 記錄特徵數量 (多項式展開後)
  - 記錄訓練時間
- 製作比較表格
- 繪製「階數 vs 性能指標」曲線
- **分析**:
  - 最佳階數是多少？
  - 階數過高會有什麼問題？(提示: 過擬合、計算成本)
  - 為什麼需要 Ridge 正則化？

### **實驗 3: Decision Tree 過擬合分析**

研究 Decision Tree 的 `max_depth` 參數對過擬合的影響。

**要比較的 max_depth**:
- 3, 5, 7, 10, 15, 20, None (無限制)

**要求**:
- 對每個 `max_depth`:
  - 訓練模型
  - 計算訓練集和測試集的 R²
  - 計算訓練集和測試集的 RMSE
  - 記錄樹的實際深度 (`tree_.max_depth`)
  - 記錄葉節點數量 (`tree_.node_count`)
- 繪製以下圖表:
  - `max_depth` vs 訓練/測試 R² (雙線圖)
  - `max_depth` vs 訓練/測試 RMSE (雙線圖)
- **分析**:
  - 何時開始出現過擬合？
  - 最佳的 `max_depth` 是多少？
  - 樹的複雜度如何影響模型性能？

### **實驗 4: SVR 核函數比較**

比較 Support Vector Regressor 使用不同核函數的效果。

**要比較的核函數**:
- `linear` (線性核)
- `poly` (多項式核, degree=2)
- `poly` (多項式核, degree=3)
- `rbf` (高斯核)
- `sigmoid` (Sigmoid 核)

**要求**:
- 對每個核函數:
  - 設定合適的 `C` 和 `epsilon` 參數
  - 訓練模型
  - 記錄訓練時間
  - 計算測試集 R²、RMSE、MAE
- 製作比較表格
- 繪製性能比較條形圖
- **分析**:
  - 哪個核函數最適合這個數據？
  - RBF 核為什麼通常表現較好？
  - 線性核與 linear regression 有何差異？

### **實驗 5: 樣本數量對模型性能的影響**

研究訓練樣本數量如何影響不同模型的性能。

**要測試的樣本數量**:
- 50, 100, 200, 300, 400, 500 (全部)

**要比較的模型** (選擇 3-4 個):
- Decision Tree
- Polynomial Regression (degree=2)
- Gradient Boosting
- Gaussian Process Regressor

**要求**:
- 對每個樣本數量和模型組合:
  - 從訓練集中隨機抽樣 (設定 `random_state=42`)
  - 訓練模型
  - 在固定的測試集上評估
  - 記錄訓練時間和 R²
- 繪製以下圖表:
  - 樣本數量 vs R² (多條線，每個模型一條)
  - 樣本數量 vs 訓練時間 (多條線)
- **分析**:
  - 哪些模型需要較多數據？
  - 哪些模型在小數據集上表現較好？
  - GPR 為什麼適合小數據集？
  - 訓練時間如何隨樣本數量變化？

### 💭 **思考題**

請根據本次作業的實驗結果，回答以下問題：

1. **模型選擇**:
   - 在觸媒反應系統中，為什麼非線性模型比線性模型更適合？
   - 如果數據量只有 50 個樣本，你會選擇哪個模型？為什麼？
   - 如果需要即時預測 (速度優先)，你會選擇哪個模型？

2. **過擬合問題**:
   - Decision Tree 為什麼容易過擬合？如何防止？
   - Gradient Boosting 如何透過 `learning_rate` 和 `n_estimators` 控制過擬合？
   - Polynomial Regression 在高階次時為什麼容易過擬合？

3. **模型解釋性**:
   - 哪些模型提供特徵重要性？這對化工應用有什麼價值？
   - Decision Tree 如何解釋其決策過程？
   - GPR 提供的不確定性量化有什麼實際應用價值？

4. **超參數調整**:
   - Gradient Boosting 的 `n_estimators` 越多越好嗎？為什麼？
   - SVR 的 `C` 參數控制什麼？如何選擇合適的值？
   - GPR 的 `alpha` 參數為什麼重要？

5. **計算成本**:
   - 為什麼 GPR 不適合大數據集？
   - Gradient Boosting 如何平衡準確度和計算速度？
   - 如果有 10,000 個樣本，你會如何選擇模型？

6. **化工應用**:
   - 在製程優化中，為什麼 GPR 的不確定性量化很重要？
   - Gradient Boosting 的特徵重要性如何幫助我們理解製程？
   - 如果要進行線上即時控制，應該選擇哪種模型？

7. **數據標準化**:
   - 為什麼在使用 SVR 和 GPR 時，數據標準化特別重要？
   - Decision Tree 是否需要標準化？為什麼？
   - 預測後為什麼要反標準化？

---
# 想對老師說的話